In [29]:
import snscrape.modules.twitter
import jsonlines
import json
import glob
from datetime import datetime
import subprocess
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from collections import Counter, OrderedDict

# Get Most Frequent Hastags Used In Tweets Related to Violent Civil Unrest

In [30]:
def get_protest_tweets (keywords, start_date, end_date, min_faves):
    tweets=[]
    locations=[]
    hashtags =[]
    scraper= ("{} since:{} until:{} min_faves:{} lang:en").format(keywords,start_date,end_date, min_faves)
    for tweet in snscrape.modules.twitter.TwitterSearchScraper(scraper).get_items():
        tweets.append(tweet.content)
        locations.append((json.loads(tweet.json()))['user']['location'])
        hashtags.append(tweet.hashtags)   
    print('Length of Tweets:',len(tweets))
    print('Length of Hashtags:',len(hashtags))
    print('Length of Locations:',len(locations))
    dict_items= {'Tweets' : tweets, 'Hashtags': hashtags, 'Locations': locations}
    return(dict_items)

In [31]:
def list_all_hashtags(hashtags):
    hashtags_flat=[]
    hashtags_flat = [item for sublist in hashtags for item in sublist]
    return(hashtags_flat)

In [32]:
def list_hashtags_freq(hashtags):
    hashtags_flat=[]
    hashtags_flat = [item for sublist in hashtags for item in sublist]

    counter=Counter(hashtags_flat)
    sorted_counter = OrderedDict(counter.most_common())
    keys=sorted_counter.keys()
    values=sorted_counter.values()
    keys_list=list(keys)
    values_list=list(values)

    return('#'+keys_list[0])

In [33]:
def show_hashtags_freq(hashtags):
    hashtags_flat=[]
    hashtags_flat = [item for sublist in hashtags for item in sublist]

    counter=Counter(hashtags_flat)
    sorted_counter = OrderedDict(counter.most_common())
    keys=sorted_counter.keys()
    values=sorted_counter.values()
    keys_list=list(keys)
    values_list=list(values)
    
    #graph
    headerColor = 'grey'
    rowEvenColor = 'lightgrey'
    rowOddColor = 'white'
    top5='rgb(230,245,201)'

    fig = go.Figure(data=[go.Table(
      header=dict(
        values=['<b>HASHTAG</b>','<b>FREQUENCY</b>'],
        line_color='darkslategray',
        fill_color=headerColor,
        align=['left','left'],
        font=dict(color='white', size=15)
      ),
      cells=dict(
        values=[
          keys_list, values_list],
        line_color='darkslategray',
        fill_color = [[top5]*5+[rowOddColor,rowEvenColor]*(int(len(keys_list)/2))],
        align = ['left', 'left'],
        font = dict(color = 'darkslategray', size = 14)
        ))
    ])

    fig.show()
    

In [34]:
keywords= 'brutality OR violence OR uprising OR protest OR riot OR resist OR demonstration'
start_date='2020-10-03'
end_date='2020-10-04'
min_faves='500'

tweets=get_protest_tweets(keywords, start_date, end_date, min_faves)

#list_all_hashtags(tweets['Hashtags'])
list_hashtags_freq(tweets['Hashtags'])
show_hashtags_freq(tweets['Hashtags'])

Length of Tweets: 143
Length of Hashtags: 143
Length of Locations: 143


# Use Top Hashtag To Get Top Tweets, Posters, and Replies

In [35]:
def scrape_hashtags(hashtag, start_date, end_date, min_faves):
    hashtags=[]
    scraper= ("{} since:{} until:{} min_faves:{} lang:en").format(keywords,start_date,end_date, min_faves)
    for hashtag in snscrape.modules.twitter.TwitterHashtagScraper(scraper).get_items():
        hashtags.append(json.loads(hashtag.json()))
    return(hashtags)
    

In [36]:
def scrape_top_users(hashtags, start_date, end_date, min_faves_u):
    usernames=[]
    for i in range(0,len(hashtags)):
        usernames.append('@' + str(hashtags[i]['user']['username']))
    
    for username in usernames:
        process= ("snscrape --jsonl twitter-search 'to:{} since:{} until:{} lang:en min_faves:{}' >>/home/kasey/Dropbox/UBS/Data/Users/{}.jsonl").format(username, start_date, end_date, min_faves_u, username)
        subprocess.call(process, shell = True)
    print('There are', len(usernames), 'usernames for which tweet replies were collected')
    

In [37]:
hashtag=list_hashtags_freq(tweets['Hashtags'])
start_date='2020-10-03'
end_date='2020-10-05'
min_faves='3000'


hashtags=scrape_hashtags(hashtag, start_date, end_date, min_faves)

min_faves_u = '3'

scrape_top_users(hashtags, start_date, end_date, min_faves_u)

There are 46 usernames for which tweet replies were collected


# Combine User Tweets to 1 jsonl file

In [50]:
def combine_user_files():
    result = []
    for f in glob.glob("/home/kasey/Dropbox/UBS/Data/Users/*.jsonl"):
        with open(f) as infile:
            for line in infile:
                result.append(json.loads(line))

    with open("/home/kasey/Dropbox/UBS/Data/Users/tweets.json", "w") as outfile:
        json.dump(result, outfile)

In [51]:
combine_user_files()